In [3]:
import openai 
import tiktoken 
import chromadb 
import langchain

In [4]:
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser

In [5]:
#API key
import dotenv
dotenv.load_dotenv()

True

In [6]:
# Ropo Path
repo_path = "C:\ProjectLFG\calculadora-tk-master"

In [15]:
# Load
loader = GenericLoader.from_filesystem(
    repo_path,
    glob="**/*",
    suffixes=[".sql"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=500)
)
documents = loader.load()
len(documents)

1

In [13]:
#Split 
from langchain.text_splitter import RecursiveCharacterTextSplitter
python_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON, 
                                                               chunk_size=2000, 
                                                               chunk_overlap=200)
texts = python_splitter.split_documents(documents)
len(texts)

12

In [14]:
#Retrieve
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
db = Chroma.from_documents(texts, OpenAIEmbeddings(disallowed_special=()))
retriever = db.as_retriever(
    search_type="mmr", # Also test "similarity"
    search_kwargs={"k": 8},
)

In [21]:
# Set up Chat
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain
llm = ChatOpenAI(model_name="gpt-3.5-turbo") 
memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [35]:
#Result
question = "Give me a detailed analysis of how this application works, include the appropriate code to explain the logic, and format it in a human understandable format"
result = qa(question)
result['answer']

Number of requested results 20 is greater than number of elements in index 12, updating n_results = 12


'This application is a calculator implemented using the Tkinter library in Python. It provides a graphical user interface where users can perform basic mathematical calculations.\n\nThe main logic of the application is divided into three classes: `Calculadora`, `Calculador`, and `CalculadorValidation`.\n\nThe `Calculadora` class is responsible for creating the layout of the calculator, distributing the buttons, and adding their functionality. It initializes the Tkinter window and sets its properties such as title, size, and background color. It also creates two frames, one for the input area and one for the buttons area. The `Calculadora` class also loads the settings from the `settings.json` file, which includes information about the current theme of the calculator.\n\nThe `Calculador` class performs the actual calculations. It has a `calculation` method that takes a calculation expression as input and returns the result of the calculation. This method uses the `eval` function to eval

In [36]:
#Expanded Results
print(result['answer'])

This application is a calculator implemented using the Tkinter library in Python. It provides a graphical user interface where users can perform basic mathematical calculations.

The main logic of the application is divided into three classes: `Calculadora`, `Calculador`, and `CalculadorValidation`.

The `Calculadora` class is responsible for creating the layout of the calculator, distributing the buttons, and adding their functionality. It initializes the Tkinter window and sets its properties such as title, size, and background color. It also creates two frames, one for the input area and one for the buttons area. The `Calculadora` class also loads the settings from the `settings.json` file, which includes information about the current theme of the calculator.

The `Calculador` class performs the actual calculations. It has a `calculation` method that takes a calculation expression as input and returns the result of the calculation. This method uses the `eval` function to evaluate th